In [1]:
import requests
r = requests.get("http://math-info.hse.ru/f/2018-19/nes-ds/gradebook.sqlite")
with open("gradebook.sqlite", "wb") as f:
    f.write(r.content)

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect("gradebook.sqlite")

In [5]:
c = conn.cursor()

In [6]:
import pandas as pd

In [10]:
pd.read_sql("""
SELECT name FROM sqlite_master WHERE type='table';
""", conn)

,name
0,courses
1,submissions
2,users
3,course_user
4,assignments


In [11]:
def rs(sql):
    return pd.read_sql(sql, conn)

In [12]:
rs("""
PRAGMA table_info(courses)
""")

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,code,TEXT,0,None,0
2,2,year,INTEGER,0,None,0
3,3,active,INTEGER,0,None,0


In [13]:
rs("""
SELECT * FROM courses
""")

,id,code,year,active
0,1,nes-ds,2017,0
1,2,nes-ds,2018,1
2,3,dj-prog,2018,1
3,4,python-all,2015,0
4,5,python-icef,2018,1


In [16]:
rs("""
SELECT * FROM submissions LIMIT 10
""")

,id,assignment_id,user_id,grade
0,1,1,1,6.0
1,2,2,1,5.0
2,3,3,1,NaN
3,4,2,2,3.0
4,5,3,2,2.0
5,6,4,3,10.0
6,7,2,3,23.0
7,8,4,3,15.0
8,9,5,4,4.0
9,10,4,4,5.0


In [17]:
rs("""
SELECT * FROM assignments LIMIT 10
""")

,id,course_id,code,name,max_grade
0,1,1,ps01,Intro to Python,10
1,2,2,ps01,Intro,12
2,3,2,ps02,Functions,8
3,4,1,ps01,"Python, part 1",22
4,5,2,ps03,Pandas,10
5,6,3,ps03,Classes,15
6,7,2,ps04,Advanced Python,32
7,8,3,ps01,Introduction,10
8,9,4,ps01,First class,7
9,10,5,ps01,Welcome to Python,8


In [18]:
rs("""
SELECT * FROM users LIMIT 10
""")

,id,first_name,last_name,email
0,1,Eddard,Stark,ned@winterfell.north.7k
1,2,Jon,Snow,jsnow@winterfell.north.7k
2,3,Arya,Stark,arya@winterfell.north.7k
3,4,Tyrion,Lannister,hand-of-a-queen@dragons.rules
4,5,Cersei,Lannister,queen@iron-throne.7k
5,6,Sansa,Stark,lady@winterfell.north.7k
6,7,Daenerys,Targaryen,queen@dragons.rules
7,8,Samwell,Tarly,samwell@citadel.7k
8,9,Brienne,None,brienne@tarth.7k
9,10,Ygritte,None,ygritte@wildlings.north.7k


In [26]:
rs("""
SELECT users.id as u_id, first_name, last_name, submissions.id as s_id, grade 
FROM users JOIN submissions 
ON users.id == submissions.user_id
""")

,u_id,first_name,last_name,s_id,grade
0,1,Eddard,Stark,1,6.0
1,1,Eddard,Stark,2,5.0
2,1,Eddard,Stark,3,NaN
3,2,Jon,Snow,4,3.0
4,2,Jon,Snow,5,2.0
5,3,Arya,Stark,6,10.0
6,3,Arya,Stark,7,23.0
7,3,Arya,Stark,8,15.0
8,4,Tyrion,Lannister,9,4.0
9,4,Tyrion,Lannister,10,5.0


In [33]:
rs("""
SELECT users.id as u_id, first_name, last_name, sum(grade) 
FROM users JOIN submissions 
ON users.id == submissions.user_id
GROUP BY u_id
""")

,u_id,first_name,last_name,sum(grade)
0,1,Eddard,Stark,11.0
1,2,Jon,Snow,5.0
2,3,Arya,Stark,48.0
3,4,Tyrion,Lannister,20.0
4,5,Cersei,Lannister,55.0
5,6,Sansa,Stark,32.0
6,7,Daenerys,Targaryen,46.0
7,8,Samwell,Tarly,20.0
8,10,Ygritte,None,8.0


In [34]:
rs("""
SELECT * FROM users, submissions
WHERE users.id == submissions.user_id
""")

,id,first_name,last_name,email,id,assignment_id,user_id,grade
0,1,Eddard,Stark,ned@winterfell.north.7k,1,1,1,6.0
1,1,Eddard,Stark,ned@winterfell.north.7k,2,2,1,5.0
2,1,Eddard,Stark,ned@winterfell.north.7k,3,3,1,NaN
3,2,Jon,Snow,jsnow@winterfell.north.7k,4,2,2,3.0
4,2,Jon,Snow,jsnow@winterfell.north.7k,5,3,2,2.0
5,3,Arya,Stark,arya@winterfell.north.7k,6,4,3,10.0
6,3,Arya,Stark,arya@winterfell.north.7k,7,2,3,23.0
7,3,Arya,Stark,arya@winterfell.north.7k,8,4,3,15.0
8,4,Tyrion,Lannister,hand-of-a-queen@dragons.rules,9,5,4,4.0
9,4,Tyrion,Lannister,hand-of-a-queen@dragons.rules,10,4,4,5.0


In [37]:
rs("""
SELECT * FROM users JOIN submissions
ON users.id == submissions.user_id
""")

,id,first_name,last_name,email,id,assignment_id,user_id,grade
0,1,Eddard,Stark,ned@winterfell.north.7k,1,1,1,6.0
1,1,Eddard,Stark,ned@winterfell.north.7k,2,2,1,5.0
2,1,Eddard,Stark,ned@winterfell.north.7k,3,3,1,NaN
3,2,Jon,Snow,jsnow@winterfell.north.7k,4,2,2,3.0
4,2,Jon,Snow,jsnow@winterfell.north.7k,5,3,2,2.0
5,3,Arya,Stark,arya@winterfell.north.7k,6,4,3,10.0
6,3,Arya,Stark,arya@winterfell.north.7k,7,2,3,23.0
7,3,Arya,Stark,arya@winterfell.north.7k,8,4,3,15.0
8,4,Tyrion,Lannister,hand-of-a-queen@dragons.rules,9,5,4,4.0
9,4,Tyrion,Lannister,hand-of-a-queen@dragons.rules,10,4,4,5.0


In [41]:
rs("""
SELECT * 
FROM users LEFT JOIN submissions 
ON users.id == submissions.user_id
""")

,id,first_name,last_name,email,id,assignment_id,user_id,grade
0,1,Eddard,Stark,ned@winterfell.north.7k,1.0,1.0,1.0,6.0
1,1,Eddard,Stark,ned@winterfell.north.7k,2.0,2.0,1.0,5.0
2,1,Eddard,Stark,ned@winterfell.north.7k,3.0,3.0,1.0,NaN
3,2,Jon,Snow,jsnow@winterfell.north.7k,4.0,2.0,2.0,3.0
4,2,Jon,Snow,jsnow@winterfell.north.7k,5.0,3.0,2.0,2.0
5,3,Arya,Stark,arya@winterfell.north.7k,6.0,4.0,3.0,10.0
6,3,Arya,Stark,arya@winterfell.north.7k,7.0,2.0,3.0,23.0
7,3,Arya,Stark,arya@winterfell.north.7k,8.0,4.0,3.0,15.0
8,4,Tyrion,Lannister,hand-of-a-queen@dragons.rules,9.0,5.0,4.0,4.0
9,4,Tyrion,Lannister,hand-of-a-queen@dragons.rules,10.0,4.0,4.0,5.0


In [40]:
rs("""
SELECT * 
FROM users OUTER JOIN submissions 
ON users.id == submissions.user_id
""")

DatabaseError: Execution failed on sql '
SELECT * 
FROM users OUTER JOIN submissions 
ON users.id == submissions.user_id
': RIGHT and FULL OUTER JOINs are not currently supported

In [45]:
rs("""
SELECT * FROM users
WHERE users.id NOT IN (SELECT user_id FROM submissions)
""")

,id,first_name,last_name,email
0,9,Brienne,None,brienne@tarth.7k


In [49]:
rs("""
SELECT * FROM users WHERE last_name == "Stark"
UNION ALL
SELECT * FROM users WHERE last_name == "Snow"
""")

,id,first_name,last_name,email
0,1,Eddard,Stark,ned@winterfell.north.7k
1,3,Arya,Stark,arya@winterfell.north.7k
2,6,Sansa,Stark,lady@winterfell.north.7k
3,2,Jon,Snow,jsnow@winterfell.north.7k


In [64]:
## Для каждого пользователя и каждой его домашки посчитать среднюю оценку
## по всем submission для этой домашки, учитывая, что она не может превышать
## max_grade данного assignment

In [73]:
rs("""
SELECT users.id, assign_id, first_name, last_name, avg(final_grade) 
FROM users JOIN 
(
  SELECT submissions.id, 
         assignments.id as assign_id,
         submissions.user_id as subm_id, 
         min(submissions.grade, assignments.max_grade) as final_grade 
  FROM submissions 
  JOIN assignments ON submissions.assignment_id == assignments.id
) subm_with_grades
WHERE users.id == subm_with_grades.subm_id
GROUP BY users.id, assign_id
""")

,id,assign_id,first_name,last_name,avg(final_grade)
0,1,1,Eddard,Stark,6.0
1,1,2,Eddard,Stark,5.0
2,1,3,Eddard,Stark,NaN
3,2,2,Jon,Snow,3.0
4,2,3,Jon,Snow,2.0
5,3,2,Arya,Stark,12.0
6,3,4,Arya,Stark,12.5
7,4,4,Tyrion,Lannister,3.0
8,4,5,Tyrion,Lannister,7.0
9,5,6,Cersei,Lannister,15.0


In [74]:
rs("""
select u.id,
       u.first_name, 
       u.last_name, 
       a.id, 
       avg(min(s.grade, max_grade)) as grade from users u
left join submissions s
on u.id = s.user_id
left join assignments a
on s.assignment_id = a.id
GROUP BY u.id, a.id
""")

,id,first_name,last_name,id,grade
0,1,Eddard,Stark,1.0,6.0
1,1,Eddard,Stark,2.0,5.0
2,1,Eddard,Stark,3.0,NaN
3,2,Jon,Snow,2.0,3.0
4,2,Jon,Snow,3.0,2.0
5,3,Arya,Stark,2.0,12.0
6,3,Arya,Stark,4.0,12.5
7,4,Tyrion,Lannister,4.0,3.0
8,4,Tyrion,Lannister,5.0,7.0
9,5,Cersei,Lannister,6.0,15.0


In [62]:
rs("""
select u.first_name, 
       u.last_name, 
       a.name, 
       round(min((s.grade / a.max_grade)*10,10)) as grade 
from users u
left join submissions s
on u.id = s.user_id
left join assignments a
on s.assignment_id = a.id
where grade is not null
ORDER BY name """)

,first_name,last_name,name,grade
0,Cersei,Lannister,Advanced Python,4.0
1,Cersei,Lannister,Classes,10.0
2,Cersei,Lannister,Classes,10.0
3,Ygritte,None,Classes,5.0
4,Daenerys,Targaryen,First class,10.0
5,Jon,Snow,Functions,3.0
6,Eddard,Stark,Intro,4.0
7,Jon,Snow,Intro,3.0
8,Arya,Stark,Intro,10.0
9,Eddard,Stark,Intro to Python,6.0
